In [1]:
import copy
import math
import numpy as np
import random as rd
import networkx as nx
import matplotlib.pyplot as plt
from scipy.optimize import minimize

import dwavebinarycsp
import dwave.inspector
from dwave.system import DWaveSampler, EmbeddingComposite, LeapHybridSampler
from dimod import ConstrainedQuadraticModel, Binary, quicksum

In [2]:
class Constellation:
    """
    n_sat -> number of satellites in the constellation
    n_dir -> number of directions a satellite can perform
    radius -> a satellite withing the range is considered a neighbor
    min_distance -> minimum distance between satellites
    max_distance -> maximum distance between satellites
    """

    def __init__(self, n_sat=10, n_dir=3, radius=1, min_distance=0.1, max_distance=2, custom_const=False) -> None:
        self.n_sat = len(custom_const) if custom_const else n_sat
        self.n_dir = n_dir
        self.radius = radius
        self.min_distance = min_distance
        self.max_distance = max_distance
        self.custom_const = custom_const  
        self.const_graph = nx.Graph()  
        self.qubits_graph = nx.Graph()
        # idx*self.n_dir+dir

        if self.custom_const:
            for idx, node in enumerate(self.custom_const):
                self.const_graph.add_node(str(idx),                                                             # '0'
                                          pos=node,                                                             # [.., ..]
                                          pos_x=node[0], pos_y=node[1],                                         # .., ..
                                          )
                for dir in range(self.n_dir):
                    self.qubits_graph.add_node(str(idx)+"-"+str(dir),
                                               q_binary = Binary(str(idx)+"-"+str(dir)))
        else:
            for idx in range(n_sat):
                while True:
                    pos_x = rd.random()*self.n_sat
                    pos_y = rd.random()*self.n_sat
                    if self._check_min_distance(pos_x, pos_y, self.min_distance):
                        self.const_graph.add_node(str(idx), 
                                                pos=[pos_x, pos_y], 
                                                pos_x=pos_x, pos_y=pos_y,
                                                )
                        for dir in range(self.n_dir):
                            self.qubits_graph.add_node(str(idx)+"-"+str(dir),
                                                       q_binary = Binary(str(idx)+"-"+str(dir)))
                        break

        # self.satellite_distances = {}
        # self.couplers_constellation = {}
        # for sat_idx_1, sat_1 in self.constellation.items():
        #     for sat_idx_2, sat_2 in self.constellation.items():
        #         sat_str_1_2 = sat_idx_1 + '-' + sat_idx_2
        #         sat_str_2_1 = sat_idx_2 + '-' + sat_idx_1
        #         distance = ((sat_1.get_pos_x() - sat_2.get_pos_x())**2 + (sat_1.get_pos_y() - sat_2.get_pos_y())**2)**0.5
        #         if distance <= self.radius and sat_1 != sat_2 and (sat_str_1_2 not in self.satellite_distances) and (sat_str_2_1 not in self.satellite_distances):
        #             self.satellite_distances[sat_str_1_2] = distance
        #             self.satellite_distances[sat_str_2_1] = distance
        #             self.const_graph.add_edge(int(sat_idx_1), int(sat_idx_2), weight=distance)
        #             self.const_graph.add_edge(int(sat_idx_2), int(sat_idx_1), weight=distance)

        #####
        for sat_1 in self.const_graph.nodes():
            for sat_2 in self.const_graph.nodes():
                distance = ((self.const_graph.nodes[sat_1]['pos_x'] - self.const_graph.nodes[sat_2]['pos_x'])**2 + (self.const_graph.nodes[sat_1]['pos_y'] - self.const_graph.nodes[sat_2]['pos_y'])**2)**0.5
                if distance <= self.radius and sat_1 != sat_2 and (not self.const_graph.has_edge(sat_1, sat_2)):
                    self.const_graph.add_edge(sat_1, sat_2, weight=distance)
        #####

        # for qubit_1 in range(self.n_dir * self.n_sat):
        #     for qubit_2 in range(qubit_1, self.n_dir * self.n_sat):
        #         qubit_couple_str_1_2 = str(qubit_1) + '-' + str(qubit_2)
        #         qubit_couple_str_2_1 = str(qubit_2) + '-' + str(qubit_1)
        #         sat_1 = str(qubit_1 // n_dir)
        #         sat_2 = str(qubit_2 // n_dir)
        #         if qubit_1 == qubit_2:
        #             self.couplers_constellation[qubit_couple_str_1_2] = 100
        #         elif qubit_2 < qubit_1 + self.n_dir:
        #             self.couplers_constellation[qubit_couple_str_1_2] = 100
        #             self.couplers_constellation[qubit_couple_str_2_1] = 100
        #         elif (sat_1 + '-' + sat_2 in self.satellite_distances) or ((sat_2 + '-' + sat_1 in self.satellite_distances)):
        #             qubit_sat_1 = qubit_1 % n_dir
        #             qubit_sat_2 = qubit_2 % n_dir
        #             new_sat_1_pos_x = self.constellation[sat_1].get_pos_x() + math.cos(qubit_sat_1*2*math.pi/n_dir)
        #             new_sat_1_pos_y = self.constellation[sat_1].get_pos_y() + math.sin(qubit_sat_1*2*math.pi/n_dir)
        #             new_sat_2_pos_x = self.constellation[sat_2].get_pos_x() + math.cos(qubit_sat_2*2*math.pi/n_dir)
        #             new_sat_2_pos_y = self.constellation[sat_2].get_pos_y() + math.sin(qubit_sat_2*2*math.pi/n_dir)
        #             new_distance = (((new_sat_1_pos_x - new_sat_2_pos_x)**2 + (new_sat_1_pos_y - new_sat_2_pos_y)**2)**0.5 - self.max_distance)**2
        #             self.couplers_constellation[qubit_couple_str_1_2] = new_distance
        #             self.couplers_constellation[qubit_couple_str_2_1] = new_distance
        #         else:
        #             self.couplers_constellation[qubit_couple_str_1_2] = 100
        #             self.couplers_constellation[qubit_couple_str_2_1] = 100

        #####
        for qubit_1 in range(self.n_dir * self.n_sat):
            node_1 = str(qubit_1 // self.n_dir)
            qubit_node_1 = qubit_1 % self.n_dir
            for qubit_2 in range(qubit_1, self.n_dir * self.n_sat):
                node_2 = str(qubit_2 // self.n_dir)
                qubit_node_2 = qubit_2 % self.n_dir
                if qubit_1 == qubit_2:
                    pass
                elif qubit_2 < qubit_1 + self.n_dir:
                    self.qubits_graph.add_edge(node_1+"-"+str(qubit_node_1), node_2+"-"+str(qubit_node_2), weight=0)
                elif self.const_graph.has_edge(node_1, node_2) or not self.const_graph.has_edge(node_2, node_1):
                    new_node_1_pos_x = self.const_graph.nodes[node_1]['pos_x'] + math.cos(qubit_node_1*2*math.pi/self.n_dir)
                    new_node_1_pos_y = self.const_graph.nodes[node_1]['pos_y'] + math.sin(qubit_node_1*2*math.pi/self.n_dir)
                    new_node_2_pos_x = self.const_graph.nodes[node_2]['pos_x'] + math.cos(qubit_node_2*2*math.pi/self.n_dir)
                    new_node_2_pos_y = self.const_graph.nodes[node_2]['pos_y'] + math.sin(qubit_node_2*2*math.pi/self.n_dir)
                    new_distance = (((new_node_1_pos_x - new_node_2_pos_x)**2 + (new_node_1_pos_y - new_node_2_pos_y)**2)**0.5 - self.max_distance)**2
                    self.qubits_graph.add_edge(node_1+"-"+str(qubit_node_1), node_2+"-"+str(qubit_node_2), weight=new_distance) 
                    
    def const_graph_comp(self):

        return self.const_graph
    
    def qubits_graph_comp(self):

        return self.qubits_graph
    
    def _check_min_distance(self, pos_x, pos_y, min_distance):
        for node in self.const_graph.nodes():
            distance = ((self.const_graph.nodes[node]['pos_x'] - pos_x)**2 + (self.const_graph.nodes[node]['pos_y'] - pos_y)**2)**0.5
            if distance < min_distance:
                return False
        return True

    def draw_constellation(self):
        for node in self.const_graph.nodes():
            plt.scatter(self.const_graph.nodes[node]['pos_x'], self.const_graph.nodes[node]['pos_y'], color='skyblue', s=200)
            plt.text(self.const_graph.nodes[node]['pos_x'], self.const_graph.nodes[node]['pos_y'], node, ha='center', va='center', color='black', fontsize=10)

        seen = []
        for edge in self.const_graph.edges():
            node_1 = str(edge[0])
            node_2 = str(edge[1])
            if ((node_1, node_2) or (node_2, node_1)) not in seen:
                seen.append((node_1, node_2))
                seen.append((node_2, node_1))
                plt.plot([self.const_graph.nodes[node_1]['pos_x'], self.const_graph.nodes[node_2]['pos_x']], [self.const_graph.nodes[node_1]['pos_y'], self.const_graph.nodes[node_2]['pos_y']], color='black', linewidth=0.1)
                plt.text((self.const_graph.nodes[node_1]['pos_x']+self.const_graph.nodes[node_2]['pos_x'])/2, (self.const_graph.nodes[node_1]['pos_y']+self.const_graph.nodes[node_2]['pos_y'])/2, str(round(self.const_graph.edges[edge]['weight'],2)), ha='center', va='center', color='black', fontsize=10)

        plt.show()

    def draw_qubit_constellation(self):
        for node in self.qubits_graph.nodes():
            plt.scatter(self.qubits_graph.nodes[node]['pos_x'], self.qubits_graph.nodes[node]['pos_y'], color='skyblue', s=200)
            plt.text(self.qubits_graph.nodes[node]['pos_x'], self.qubits_graph.nodes[node]['pos_y'], node, ha='center', va='center', color='black', fontsize=10)

        seen = []
        for edge in self.qubits_graph.edges():
            node_1 = str(edge[0])
            node_2 = str(edge[1])
            if ((node_1, node_2) or (node_2, node_1)) not in seen:
                seen.append((node_1, node_2))
                seen.append((node_2, node_1))
                plt.plot([self.qubits_graph.nodes[node_1]['pos_x'], self.qubits_graph.nodes[node_2]['pos_x']], [self.qubits_graph.nodes[node_1]['pos_y'], self.qubits_graph.nodes[node_2]['pos_y']], color='black', linewidth=0.1)

        plt.show()

    def fake_bitstring_gen(self):
        return [rd.choice([0, 1]) for _ in range(self.n_sat * self.n_dir)]
    
    def step(self, bitstring):
        # for idx, sat in self.constellation.items():
        #     sat_bitstr = bitstring[(int(idx)*self.n_dir) : (int(idx)*self.n_dir+self.n_dir)]
        #     sat.move(sat_bitstr, self.n_dir)

        #####
        for node in self.const_graph.nodes():
            sat_bitstr = bitstring[(int(node)*self.n_dir) : (int(node)*self.n_dir+self.n_dir)]
            for dir in range(self.n_dir):
                self.const_graph.nodes[node]['pos'][0] = self.const_graph.nodes[node]['pos'][0] + sat_bitstr[dir]*math.cos(dir*2*math.pi/self.n_dir)
                self.const_graph.nodes[node]['pos'][1] = self.const_graph.nodes[node]['pos'][1] + sat_bitstr[dir]*math.sin(dir*2*math.pi/self.n_dir)
                self.const_graph.nodes[node]['pos_x'] = self.const_graph.nodes[node]['pos_x'] + sat_bitstr[dir]*math.cos(dir*2*math.pi/self.n_dir)
                self.const_graph.nodes[node]['pos_y'] = self.const_graph.nodes[node]['pos_y'] + sat_bitstr[dir]*math.sin(dir*2*math.pi/self.n_dir)
        
        for edge in self.qubits_graph.edges():
            node_1, qubit_node_1 = edge[0][0], int(edge[0][-1])
            node_2, qubit_node_2 = edge[1][0], int(edge[1][-1])
            
            new_node_1_pos_x = self.const_graph.nodes[node_1]['pos_x'] + math.cos(qubit_node_1*2*math.pi/self.n_dir)
            new_node_1_pos_y = self.const_graph.nodes[node_1]['pos_y'] + math.sin(qubit_node_1*2*math.pi/self.n_dir)
            new_node_2_pos_x = self.const_graph.nodes[node_2]['pos_x'] + math.cos(qubit_node_2*2*math.pi/self.n_dir)
            new_node_2_pos_y = self.const_graph.nodes[node_2]['pos_y'] + math.sin(qubit_node_2*2*math.pi/self.n_dir)
            new_distance = (((new_node_1_pos_x - new_node_2_pos_x)**2 + (new_node_1_pos_y - new_node_2_pos_y)**2)**0.5 - self.max_distance)**2
            self.qubits_graph.edges[edge]['weights'] = new_distance

            # TODO: da aggiungere la dinamicità delle connessioni

        return self.qubits_graph
        #####

In [14]:
def graph_to_cqm(const, n_dir, constraints):

    const_graph = const.const_graph_comp()
    qubits_graph = const.qubits_graph_comp()

    cqm = ConstrainedQuadraticModel()

    sum = 0
    for edge in qubits_graph.edges():
            qubit_1, qubit_2 = edge[0], edge[1]
            cqm.add_discrete(qubits_graph.nodes[qubit_1]['q_binary'])
            cqm.add_discrete(qubits_graph.nodes[qubit_2]['q_binary'])
            weight = qubits_graph.edges[edge]['weight']
            sum = sum + qubits_graph.nodes[qubit_1]['q_binary'] * qubits_graph.nodes[qubit_2]['q_binary'] * weight

    cqm.set_objective(sum)

    if constraints:
        for node in const_graph.nodes():
            constraint = 0
            for dir in range(n_dir):
                constraint = constraint + qubits_graph.nodes[node+"-"+str(dir)]['q_binary']
            cqm.add_constraint(constraint <= n_dir-1)

    return cqm

In [4]:
n_sat = 10                       # how many satellites
radius = 5                      # Within this radius, a satellite is considered a neighboring
n_dir = 3                       # how many dicrections a satellite can perform
min_distance = 1                # minimum initial distance between two satellites
max_distance = 2                # maximum distance in order to have a connection
epochs = 100
constraints = True
custom_const = [[0,0], [0,1], [1,0], [1,1]]

# const = Constellation(n_sat, n_dir, radius, min_distance, max_distance)
const = Constellation(n_sat, n_dir, radius, min_distance, max_distance, custom_const)

for _ in range(epochs):

    const.draw_constellation()
    # const.draw_qubit_constellation()

    cqm_model = graph_to_cqm(const, n_dir, constraints)
    # sampler = DWaveSampler()
    # bitstring = sampler.sample_cqm(cqm_model, num_reads=10)
    # const.step(bitstring.first.sample)

    bitstring = const.fake_bitstring_gen()

    qubits_graph = const.step(bitstring)
